In [1]:
from signals import loadData
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [25]:
import strategies
test_data = loadData(287*180, False)[287*16:287*18]
sma = strategies.SMA(10, 50, is_statefull= False)
rsi = strategies.RSI(14, is_statefull= False)
macd = strategies.MACD(12, 26, 9, is_statefull= False)
composite = strategies.StateFullComposite(10000, strategies=[sma, rsi, macd])
actions = []
amounts = []
for i in range(1, len(test_data)+1):
    action, amount = composite.action( test_data.head(i) )
    actions.append( action )
    amounts.append( amount )
test_data['signal'] = actions
test_data['amount'] = [ abs(amount) for amount in amounts ]

test_data['signal'].replace({0:'hold', 1: 'buy', -1:'sell', 2: 'strong_buy', 3:'very_strong_buy', -2: 'strong_sell', -3: 'very_strong_sell'}, inplace=True)
is_signal_col = lambda name: 'mid' in name or 'quote' in name or 'amount' in name or 'RSI' in name or 'MACD' in name or 'Signal' in name
line = px.line(test_data, x = 'timeStamp', y = [col for col in test_data.columns if is_signal_col(col)])
scatter = px.scatter(test_data[~test_data['signal'].isin(['hold'])], x = 'timeStamp', y='mid', color='signal', size = 'mid', hover_data=['amount'] )
plot = go.Figure(data=line.data + scatter.data)
plot.show()

bought 2852 units at 1.09922/unit
total holding: 2852 at avg buy price: 1.09922
total profit: 0
sold 2252 units at profit: 0.0014699999999998603 avg buy price: 1.09922 current price: 1.10069
total holding: 600 at avg buy price: 1.0937026000000007
total profit: 3.3104399999996854
sold 600 units at profit: 0.006557399999999269 avg buy price: 1.0937026000000007 current price: 1.10026
total holding: 0 at avg buy price: 0
total profit: 7.244879999999247
bought 2490 units at 1.09812/unit
total holding: 2490 at avg buy price: 1.09812
total profit: 7.244879999999247
bought 1769 units at 1.09794/unit
total holding: 4259 at avg buy price: 1.0980452359708852
total profit: 7.244879999999247
bought 680 units at 1.09845/unit
total holding: 4939 at avg buy price: 1.0981009637578458
total profit: 7.244879999999247
sold 1077 units at profit: 0.0010790362421542188 avg buy price: 1.0981009637578458 current price: 1.09918
total holding: 3862 at avg buy price: 1.097800051786639
total profit: 8.407002032799

In [24]:
test_data = loadData(287*180, False)[287*16:287*18]
is_signal_col = lambda name: 'mid' in name
line = px.line(test_data, x = 'timeStamp', y = [col for col in test_data.columns if is_signal_col(col)])
line.show()